# Query Text Data

Demonstrates use of the Intelligence Toolkit library to respond to questions about a collection of text documents.

See [readme](https://github.com/microsoft/intelligence-toolkit/blob/main/app/workflows/query_text_data/README.md) for more details.


In [1]:
import sys
sys.path.append("..")
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU' # Avoids threadpoolctl error in Linux and MacOS
from toolkit.query_text_data import QueryTextData, ProcessedChunks, ChunkSearchConfig, AnswerConfig, AnswerObject
from toolkit.AI import OpenAIConfiguration, OpenAIEmbedder
from toolkit.helpers.constants import CACHE_PATH
import nest_asyncio # Necessary to run async code in ipynb
import pandas as pd
nest_asyncio.apply()

c:\Users\daedge\AppData\Local\pypoetry\Cache\virtualenvs\intelligence-toolkit-lXFNld9n-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
# Create the workflow object
qtd = QueryTextData()
# Set up the AI model and embedding model
ai_configuration = OpenAIConfiguration(
    {
        "api_type": "OpenAI",
        "api_key": os.environ["OPENAI_API_KEY"],
        "model": "gpt-4o",
    }
)
qtd.set_ai_config(
    ai_configuration=ai_configuration,
    embedding_cache=CACHE_PATH
)
text_embedder = OpenAIEmbedder(
    configuration=ai_configuration,
)
qtd.set_embedder(text_embedder)
print("Created QueryTextData object")

Created QueryTextData object


In [3]:
# Provide text inputs as a dictionary of title->text
# Enter the path to your own data here
input_path = "../example_outputs/query_text_data/news_articles/news_articles_texts.csv"
file_name = input_path.split("/")[-1]
df = pd.read_csv(input_path)
text_to_chunks = qtd.process_data_from_df(df, file_name)
print("Processed data from df")

Processed data from df


In [4]:
# Process the chunks into the index data structures
processed_chunks: ProcessedChunks = qtd.process_text_chunks()
print(f"Processed chunks")

Processed chunks


In [5]:
# Embed the text chunks
cid_to_vector = await qtd.embed_text_chunks()
print(f"Embedded chunks")

Got 501 existing texts
Got 0 new texts
Embedded chunks


In [6]:
# Edit the question to be answered
question = "What events are discussed?"
# Mine relevant chunks to the question
chunk_search_config: ChunkSearchConfig = ChunkSearchConfig(
    # How many relevance tests are permitted per query. Higher values may provide higher quality results at higher cost
    relevance_test_budget=50,
    # How many chunks before and after each relevant chunk to test, once the relevance test budget is near or the search process has terminated
    adjacent_test_steps=1, 
    # How many relevance tests to run on each community in turn
    community_relevance_tests=5,
    # How many relevance tests to run in parallel at a time
    relevance_test_batch_size=5,
    # How many chunks to use to rank communities by relevance
    community_ranking_chunks=5,
    # When to restart testing communities in relevance order
    irrelevant_community_restart=5
)
relevant_cids, search_summary = await qtd.detect_relevant_text_chunks(
    question=question,
    chunk_search_config=chunk_search_config
)
print(f"Mined relevant chunks")

Top semantic search cids: [213, 104, 50, 38, 449, 76, 178, 182, 29, 359, 344, 484, 376, 314, 464, 381, 399, 181, 93, 324, 179, 150, 202, 409, 144, 155, 441, 96, 279, 64, 229, 237, 56, 189, 55, 364, 370, 219, 251, 400, 388, 84, 62, 341, 284, 130, 47, 485, 277, 31, 183, 310, 196, 239, 267, 475, 290, 470, 457, 138, 440, 112, 296, 446, 164, 180, 24, 418, 224, 493, 4, 468, 168, 173, 380, 244, 188, 90, 233, 382, 65, 349, 442, 348, 14, 304, 109, 337, 105, 379, 391, 408, 166, 443, 165, 216, 72, 374, 20, 152]
Level 0 community sequence: ['1.1', '1.6', '1.10', '1.7', '1.3', '1.4', '1.14', '1.8', '1.18', '1.5', '1.12', '1.11', '1.9', '1.17', '1.19', '1.2', '1.16', '1.13', '1.15']
Level 1 community sequence: ['1.1.1', '1.1.5', '1.1.2', '1.1.4', '1.10.1', '1.1.3', '1.6.3', '1.6.2', '1.1.6', '1.6.4', '1.6.1', '1.18', '1.14.2', '1.3.2', '1.3.3', '1.10.2', '1.1.7', '1.1.8', '1.14.4', '1.4.1', '1.5.2', '1.8.4', '1.10.3', '1.3.4', '1.9.1', '1.17.3', '1.12.5', '1.10.5', '1.7.25', '1.7.12', '1.4.3', '1.11

100%|██████████| 5/5 [00:00<00:00,  5.05it/s]


Community 1 relevant? True
Incrementing level
New level 0 loop after 5 tests
Community sequence: ['1.1', '1.6', '1.10', '1.7', '1.3', '1.4', '1.14', '1.8', '1.18', '1.5', '1.12', '1.11', '1.9', '1.17', '1.19', '1.2', '1.16', '1.13', '1.15']
Assessing relevance for community 1.1 with chunks [178, 182, 29, 359, 344]


100%|██████████| 5/5 [00:00<00:00,  5.14it/s]


Community 1.1 relevant? True
Assessing relevance for community 1.6 with chunks [76, 93, 184, 261, 458]


...

Community 1.9 relevant? False
Assessing relevance for community 1.17 with chunks [374, 58, 301, 40, 488]


0it [00:00, ?it/s]


Community 1.17 relevant? False
Assessing relevance for community 1.2 with chunks [266, 151, 472, 435, 185]


0it [00:00, ?it/s]

Community 1.2 relevant? False
0 successive irrelevant communities; restarting
Incrementing level
Mined relevant chunks


In [7]:
# Generate an extended answer to the question, which could then be summarized into a shorter form
answer_config = AnswerConfig(
    extract_claims = False, # Whether to extract claims from relevant text chunks
    claim_search_depth = 0, # If extracting claims, how many chunks to search for additional claim support
    target_chunks_per_cluster = 5, # How many chunks to aim to analyze together in a single LLM call
)
answer_object: AnswerObject = await qtd.answer_question_with_relevant_chunks(
    answer_config=answer_config
)
print(f"Answered question")

c:\Users\daedge\AppData\Local\pypoetry\Cache\virtualenvs\intelligence-toolkit-lXFNld9n-py3.11\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
100%|██████████| 10/10 [00:26<00:00,  2.61s/it]


Answered question


In [8]:
# Output the final extended answer
print(answer_object.extended_answer)

# Diverse Global and Local Events Highlight Cultural, Environmental, and Community Engagement

*In response to: What events are discussed?*

## Executive summary

The report covers a wide range of events that emphasize cultural, environmental, and community engagement across the globe. These events include international forums addressing climate change and economic challenges, music and cultural festivals that enhance local and global cultural landscapes, and community-driven initiatives that foster local engagement and support. Additionally, the report highlights efforts in mental health advocacy, education reform, and technological innovations, showcasing a multifaceted approach to addressing contemporary global and local issues.

## Theme: International Forums Address Climate and Economic Challenges

This theme explores global forums that focus on climate change solutions and economic challenges, emphasizing the need for international cooperation and innovative strategies to address